In [ ]:
import pandas as pd
from scipy.io import loadmat
import altair as alt
import numpy as np
import math
from scipy.interpolate import InterpolatedUnivariateSpline
from src.spinorama.load import graph_melt
from src.spinorama.graph import graph_freq


alt.data_transformers.disable_max_rows()

In [ ]:
#h_mat = loadmat("datas/Princeton/Genelec 8351A/Genelec8351A_H_IR.mat")
#v_mat = loadmat("datas/Princeton/Genelec 8351A/Genelec8351A_V_IR.mat")
#h_mat = loadmat("datas/Princeton/Genelec 8030A/Genelec_H_IR.mat")
#v_mat = loadmat("datas/Princeton/Genelec 8030A/Genelec_V_IR.mat")
#h_mat = loadmat("datas/Princeton/KEF LS50/KEF50_H_IR.mat")
#v_mat = loadmat("datas/Princeton/KEF LS50/KEF50_V_IR.mat")
h_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_H_IR.mat")
v_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_V_IR.mat")

In [ ]:
timestep = 1./h_mat['fs_H']
freq = np.fft.fftfreq(2**14, d=timestep)
print(freq.size)
print(freq)

In [ ]:
def parse_graph_freq_princeton_mat(mat, suffix):
    """ Suffix can be either H or V """
    ir_name = 'IR_{:1s}'.format(suffix)
    fs_name = 'fs_{:1s}'.format(suffix)
    # compute Freq                                                                                                                   
    timestep = 1./mat[fs_name]
    # hummm                                                                                                                          
    freq = np.fft.fftfreq(2**14, d=timestep)
    # reduce spectrum to 0 to 24kHz                                                                                                  
    # lg = 2**14                                                                                                                     
    # 24k is lgs = int(lg/4)                                                                                                         
    # 20k is at 3414                                                                                                                 
    lgs = 3414
    xs = freq[0][0:lgs]
    #    
    df = pd.DataFrame({'Freq': xs})
    dfy = np.empty(0, dtype=float)
    # loop over measurements                                                                                                         
    for i in range(0, 72):
        # extract ir                                                                                                                 
        ir = mat[ir_name][i]
        # compute FFT                                                                                                                
        y = np.fft.fft(ir)
        ys = np.abs(y[0:lgs])
        # check for 0 (from manual: 0 means not measured)                                                                            
        if ys.max() == 0.0:
            continue
        # apply formula from paper to translate to dbFS                                                                              
        ys = 105.+np.log10(ys)*20.
        # interpolate to smooth response                                                                                             
        # s = InterpolatedUnivariateSpline(xs, ys)                                                                                   
        # pretty print label, per 5 deg increment, follow klippel labelling                                                          
        ilabel =i*5
        if ilabel > 180: 
            ilabel = ilabel-360
        label = '{:d}°'.format(ilabel)
        if ilabel == 0:
            label = 'On Axis'
        df[label] = ys
    return df

h_spl_unmelted = parse_graph_freq_princeton_mat(h_mat, 'H')
v_spl_unmelted = parse_graph_freq_princeton_mat(v_mat, 'V')
print(v_spl_unmelted)

h_spl = graph_melt(h_spl_unmelted)
v_spl = graph_melt(v_spl_unmelted)

h_chart = alt.Chart(h_spl).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[40,105])),
    alt.Color('Measurements:N', sort=None)
).properties(
    width=400,
    height=400
)

v_chart = alt.Chart(v_spl).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[40,105])),
    alt.Color('Measurements:N', sort=None)
).properties(
    width=400,
    height=400
)

h_chart | v_chart

In [ ]:
print(h_spl_unmelted.shape)
print(v_spl_unmelted.shape)
print('H',h_spl_unmelted.columns)
print('V',v_spl_unmelted.columns)

In [ ]:
#print(h_spl_unmelted.columns)
#print(v_spl_unmelted.columns)

def spatial_average1(window, sel):
    window_sel = window[[c for c in window.columns if c in sel and c != 'Freq']]
    return pd.DataFrame({
        'Freq': window.Freq,
        'dB': window_sel.mean(axis=1)
    })


def spatial_average2(h_df, h_sel, v_df, v_sel):
    # some graphs don't have all angles                                                                             
    h_df_sel = h_df[[c for c in h_df.columns if c in h_sel]]
    v_df_sel = v_df[[c for c in v_df.columns if c in v_sel]]
    # some don't have vertical measurements
    if len(v_df_sel.columns) == 1:
        return spatial_average1(h_df, h_sel)
    # merge both                                                                                                    
    window = h_df_sel.merge(v_df_sel, left_on='Freq', right_on='Freq', suffixes=('_h', '_v'))
    return pd.DataFrame({
        'Freq': window.Freq,
        'dB': window.loc[:, lambda df: [c for c in df.columns if c != 'Freq']].mean(axis=1)
    })


listening_window = spatial_average2(
    h_spl_unmelted, ['Freq', '10°', '20°', '30°'], 
    v_spl_unmelted, ['Freq', 'On Axis', '10°', '-10°'])

floor_bounce = spatial_average1(
    v_spl_unmelted, ['Freq', '-20°',  '-30°', '-40°'])

ceiling_bounce = spatial_average1(
    v_spl_unmelted, ['Freq', '40°',  '50°', '60°'])

front_wall_bounce = spatial_average1(
    h_spl_unmelted, ['Freq', 'On Axis', '10°',  '20°', '30°'])

side_wall_bounce = spatial_average1(
    h_spl_unmelted, ['Freq', '40°',  '50°',  '60°',  '70°',  '80°'])

rear_wall_bounce = spatial_average1(
    h_spl_unmelted, ['Freq', '90°',  '180°'])


early_reflections = graph_melt(pd.DataFrame({
    'Freq': listening_window.Freq,
    'H On Axis': h_spl_unmelted.loc[:, 'On Axis'],
    'V On Axis': v_spl_unmelted.loc[:, 'On Axis'],
    'Floor Bounce': floor_bounce.dB,
    'Ceiling Bounce': ceiling_bounce.dB,
    'Front Wall Bounce': front_wall_bounce.dB,
    'Side Wall Bounce': side_wall_bounce.dB,
    'Rear Wall Bounce': rear_wall_bounce.dB,
}))


floor_reflection = spatial_average1(
    v_spl_unmelted, ['Freq', '-20°',  '-30°', '-40°'])

ceiling_reflection = spatial_average1(
    v_spl_unmelted, ['Freq', '40°',  '50°', '60°'])


vertical_reflections = graph_melt(pd.DataFrame({
    'Freq': listening_window.Freq,
    'H On Axis': h_spl_unmelted.loc[:, 'On Axis'],
    'V On Axis': v_spl_unmelted.loc[:, 'On Axis'],
    'Floor Reflection': floor_reflection.dB,
    'Ceiling Reflection': ceiling_reflection.dB,
}))

front = spatial_average1(
    h_spl_unmelted, ['Freq', 'On Axis', '10°',  '20°', '30°'])

side = spatial_average1(
    h_spl_unmelted, ['Freq', '40°',  '50°', '60°', '70°', '80°'])

rear = spatial_average1(
    h_spl_unmelted, ['Freq', '90°',  '100°', '110°', '120°', '130°',
                     '140°', '150°', '160°', '170°', '180°'])


# Horizontal Reflections
# Front: 0°, ± 10o, ± 20o, ± 30o horizontal
# Side: ± 40°, ± 50°, ± 60°, ± 70°, ± 80° horizontal
# Rear: ± 90°, ± 100°, ± 110°, ± 120°, ± 130°, ± 140°, ± 150°, ± 160°, ± 170°, 180°
# horizontal, (i.e.: the horizontal part of the rear hemisphere).

horizontal_reflections = graph_melt(pd.DataFrame({
    'Freq': listening_window.Freq,
    'H On Axis': h_spl_unmelted.loc[:, 'On Axis'],
    'V On Axis': v_spl_unmelted.loc[:, 'On Axis'],
    'Front': front.dB,
    'Side': side.dB,
    'Rear': rear.dB
}))


early_reflections_bounce = spatial_average2(
    h_spl_unmelted, ['Freq', 'On Axis', '10°',  '20°', '30°', '40°',  '50°',  '60°',  '70°',  '80°', '90°',  '180°'],
    v_spl_unmelted, ['Freq', 'On Axis', '10°', '-10°', '-20°',  '-30°', '-40°', '40°',  '50°', '60°']
)

# Early Reflections Directivity Index (ERDI)
# The Early Reflections Directivity Index is defined as the difference 
# between the listening window curve and the early reflections curve.
# add 60 (graph start at 60)
early_reflections_directivity_index = listening_window.dB - early_reflections_bounce.dB + 60


early_reflections = graph_melt(pd.DataFrame({
    'Freq': listening_window.Freq,
    'H On Axis': h_spl_unmelted.loc[:, 'On Axis'],
    'V On Axis': v_spl_unmelted.loc[:, 'On Axis'],
    'Floor Bounce': floor_bounce.dB,
    'Ceiling Bounce': ceiling_bounce.dB
}))

from math import log10, pow

# weigth http://emis.impa.br/EMIS/journals/BAG/vol.51/no.1/b51h1san.pdf
sp_weigths = {
    'On Axis': 0.000604486,
    '10°': 0.004730189,
    '20°': 0.008955027,
    '30°': 0.012387354,
    '40°': 0.014989611,
    '50°': 0.016868154,
    '60°': 0.018165962,
    '70°': 0.019006744,
    '80°': 0.019477787,
    '90°': 0.019629373,
   '100°': 0.019477787,
   '110°': 0.019006744,
   '120°': 0.018165962,
   '130°': 0.016868154,
   '140°': 0.014989611,
   '150°': 0.012387354,
   '160°': 0.008955027,
   '170°': 0.004730189,
   '180°': 0.000604486,
  '-170°': 0.004730189,
  '-160°': 0.008955027,
  '-150°': 0.012387354,
  '-140°': 0.014989611,
  '-130°': 0.016868154,
  '-120°': 0.018165962,
  '-110°': 0.019006744,
  '-100°': 0.019477787,
   '-90°': 0.019629373,
   '-80°': 0.019477787,
   '-70°': 0.019006744,
   '-60°': 0.018165962,
   '-50°': 0.016868154,
   '-40°': 0.014989611,
   '-30°': 0.012387354,
   '-20°': 0.008955027,
   '-10°': 0.004730189,
}

# Sound Power
# The sound power is the weighted rms average of all 70 measurements,
# with individual measurements weighted according to the portion of the 
# spherical surface that they represent. Calculation of the sound power 
# curve begins with a conversion from SPL to pressure, a scalar magnitude. 
# The individual measures of sound pressure are then weighted according 
# to the values shown in Appendix C and an energy average (rms) is 
# calculated using the weighted values. The final average is converted 
# to SPL.
def trim(c):
    if c[-2:] == '_v' or c[-2:] == '_h':
        #print('trim '+c+' '+c[:-2])
        return c[:-2]
    return c

def valid(c):
    if c[0] == 'O':
        #print('c is O')
        return True
    elif c[0] == 'F':
        return False
    elif int(trim(c)[:-1]) % 10 == 0:
        #print('c is 10')
        return True
    return False

sp_window = h_spl_unmelted.merge(
    v_spl_unmelted, 
    left_on='Freq', right_on='Freq', suffixes=('_h', '_v')
)
sp_cols = sp_window.columns

def spl2pressure(spl : float) -> float:
    return pow(10,(spl-105)/20)

def pressure2spl(p : float) -> float:
    return 105+20*log10(p)

def rms(spl : np.array) -> float:
    avg = np.sum([sp_weigths[trim(c)]*spl2pressure(spl[c])**2 for c in sp_cols if valid(c)])
    wsm = np.sum([sp_weigths[trim(c)]                         for c in sp_cols if valid(c)])
    return pressure2spl(np.sqrt(avg/wsm))

sp_window['rms'] = sp_window.apply(rms, axis=1) 

sound_power = pd.DataFrame({
        'Freq': sp_window.Freq, 
        'dB': sp_window.rms
})

# Sound Power Directivity Index (SPDI)
# For the purposes of this standard the Sound Power Directivity Index is defined 
# as the difference between the listening window curve and the sound power curve. 
# An SPDI of 0 dB indicates omnidirectional radiation. The larger the SPDI, the 
# more directional the loudspeaker is in the direction of the reference axis.
sound_power_directivity_index = listening_window.dB - sp_window.rms + 60

cea2034 = graph_melt(pd.DataFrame({
    'Freq': listening_window.Freq,
    'On Axis (H)': h_spl_unmelted.loc[:, 'On Axis'],
    'On Axis (V)': v_spl_unmelted.loc[:, 'On Axis'],
    'Listening Window': listening_window.dB,
    'Early Reflections Directivity Index': early_reflections_directivity_index,
    'Sound Power': sound_power.dB,
    'Sound Power Directivity Index': sound_power_directivity_index,
}))


chart_cea2034 = alt.Chart(cea2034).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,95])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)


chart_early_reflections = alt.Chart(early_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,105])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)

chart_vertical_reflections = alt.Chart(vertical_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,105])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)

chart_horizontal_reflections = alt.Chart(horizontal_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,95])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)


chart_cea2034 
# chart_early_reflections
# chart_vertical_reflections
# chart_horizontal_reflections
